In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

## Data Import and preprocessing

In [4]:
data=pd.read_csv('Tweets.csv')

In [6]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
1,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)
3,5.703010e+17,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)
4,5.703010e+17,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,24-02-2015 11:13,San Francisco CA,Pacific Time (US & Canada)


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11541 entries, 0 to 11540
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      11541 non-null  float64
 1   airline_sentiment             11541 non-null  object 
 2   airline_sentiment_confidence  11541 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     9508 non-null   float64
 5   airline                       11541 non-null  object 
 6   airline_sentiment_gold        37 non-null     object 
 7   name                          11541 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 11541 non-null  int64  
 10  text                          11541 non-null  object 
 11  tweet_coord                   838 non-null    object 
 12  tweet_created                 11541 non-null  object 
 13  t

In [12]:
data_subset=data[['text','airline_sentiment']]

In [14]:
data_subset.head()

,text,airline_sentiment
0,@VirginAmerica plus you've added commercials t...,positive
1,@VirginAmerica it's really aggressive to blast...,negative
2,@VirginAmerica and it's a really big bad thing...,negative
3,@VirginAmerica seriously would pay $30 a fligh...,negative
4,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [16]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stop_words=set(stopwords.words('english'))

In [22]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [26]:
wordnet = WordNetLemmatizer()
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [28]:
data_subset.head()

,text,airline_sentiment
0,@VirginAmerica plus you've added commercials t...,positive
1,@VirginAmerica it's really aggressive to blast...,negative
2,@VirginAmerica and it's a really big bad thing...,negative
3,@VirginAmerica seriously would pay $30 a fligh...,negative
4,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [30]:
data_subset['clean_text']=data_subset['text'].apply(text_preproc)

In [34]:
data_subset.head()

,text,airline_sentiment,clean_text
0,@VirginAmerica plus you've added commercials t...,positive,plus added commercials experience tacky
1,@VirginAmerica it's really aggressive to blast...,negative,really aggressive blast obnoxious entertainme...
2,@VirginAmerica and it's a really big bad thing...,negative,really big bad thing
3,@VirginAmerica seriously would pay $30 a fligh...,negative,seriously would pay flight seats playing it r...
4,"@VirginAmerica yes, nearly every time I fly VX...",positive,yes nearly every time fly vx ear worm wont go...


## Basic ML model

In [37]:
X_train,X_test,y_train,y_test=train_test_split(data_subset['clean_text'],data_subset['airline_sentiment'],test_size=0.2,random_state=1)

In [41]:
vectorizer=TfidfVectorizer(use_idf=True,max_features=10)
X_train_tfid=vectorizer.fit_transform(X_train)
X_test_tfid=vectorizer.fit_transform(X_test)

In [43]:
classifier=MultinomialNB()
classifier.fit(X_train_tfid,y_train)

MultinomialNB()

In [47]:
y_pred=classifier.predict(X_test_tfid)

In [49]:


from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
cf_matrix=confusion_matrix(y_test,y_pred)
print(f'Confusion_matrix: {cf_matrix}')
print('**************************')
print('')
clf_report=classification_report(y_test,y_pred)
print(f'{clf_report}')
print('**************************')
print('')
Accuracy=accuracy_score(y_test,y_pred)
print(f'accuracy_score : {Accuracy}')
print('**************************')
print('')


Confusion_matrix: [[1796   41]
 [ 370  102]]
**************************

              precision    recall  f1-score   support

    negative       0.83      0.98      0.90      1837
    positive       0.71      0.22      0.33       472

    accuracy                           0.82      2309
   macro avg       0.77      0.60      0.61      2309
weighted avg       0.81      0.82      0.78      2309

**************************

accuracy_score : 0.8220008661758337
**************************



In [55]:
from sklearn.metrics import roc_auc_score,roc_curve, auc

# Assuming 'y_test' contains true labels and 'y_pred_proba' contains predicted probabilities
y_pred_proba = classifier.predict_proba(X_test_tfid)[:, 1]  # Probability of positive class
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC score: {auc:.6f}")

AUC score: 0.709011


## Hybrid Model

In [58]:
#Adding regular expression to data
import re
from collections import Counter
Counter(" ".join(data_subset['clean_text']).split()).most_common(100)

[('flight', 3315),
 ('i', 1104),
 ('get', 1102),
 ('cancelled', 956),
 ('thanks', 920),
 ('service', 909),
 ('customer', 724),
 ('time', 695),
 ('help', 686),
 ('hours', 656),
 ('hold', 621),
 ('plane', 584),
 ('amp', 582),
 ('us', 567),
 ('thank', 535),
 ('still', 529),
 ('delayed', 517),
 ('can', 502),
 ('one', 492),
 ('flights', 478),
 ('call', 477),
 ('gate', 476),
 ('hour', 467),
 ('flightled', 460),
 ('you', 456),
 ('bag', 448),
 ('back', 448),
 ('would', 438),
 ('got', 399),
 ('late', 395),
 ('phone', 392),
 ('need', 391),
 ('please', 386),
 ('airline', 374),
 ('like', 353),
 ('today', 351),
 ('waiting', 351),
 ('guys', 334),
 ('great', 317),
 ('that', 316),
 ('wait', 307),
 ('u', 306),
 ('never', 304),
 ('fly', 303),
 ('me', 303),
 ('day', 303),
 ('trying', 302),
 ('it', 291),
 ('delay', 288),
 ('really', 283),
 ('airport', 283),
 ('minutes', 278),
 ('even', 274),
 ('going', 271),
 ('people', 268),
 ('last', 262),
 ('way', 262),
 ('bags', 259),
 ('weather', 257),
 ('good', 254)

In [60]:
Postive_pattern_1 = r"thanks"
Postive_pattern_2 = r"thank"
Postive_pattern_3 = r"great"
Postive_pattern_4 = r"good"
Postive_pattern_5 = r"safe"
Postive_pattern_6 = r"awesome"
Postive_pattern_7 = r"new"
Postive_pattern_8 = r"satisfied"
Positive_Pattern_List = [Postive_pattern_1,Postive_pattern_2,Postive_pattern_3,Postive_pattern_4,
                        Postive_pattern_5,Postive_pattern_6,Postive_pattern_7,Postive_pattern_8]

In [68]:
Positive_Complex_Pattern = re.compile('|'.join(['(%s)' % i for i in Positive_Pattern_List]),re.IGNORECASE)

In [70]:
Positive_Complex_Pattern

re.compile(r'(thanks)|(thank)|(great)|(good)|(safe)|(awesome)|(new)|(satisfied)',
           re.IGNORECASE|re.UNICODE)

In [72]:
Negative_pattern_1 = r"cancelled"
Negative_pattern_2 = r"delayed"
Negative_pattern_3 = r"trying"
Negative_pattern_4 = r"please"
Negative_pattern_5 = r"wait"
Negative_pattern_6 = r"worst"
Negative_pattern_7 = r"lost"
Negative_pattern_8 = r"never"
Negative_pattern_9 = r"fraud trans"

Negative_Pattern_List = [Negative_pattern_1,Negative_pattern_2,Negative_pattern_3,Negative_pattern_4,
                        Negative_pattern_5,Negative_pattern_6,Negative_pattern_7,Negative_pattern_8]

In [74]:
Negative_Complex_Pattern = re.compile('|'.join(['(%s)' % i for i in Negative_Pattern_List]),re.IGNORECASE)

In [76]:
Negative_Complex_Pattern

re.compile(r'(cancelled)|(delayed)|(trying)|(please)|(wait)|(worst)|(lost)|(never)',
           re.IGNORECASE|re.UNICODE)

In [92]:
data_subset["Negative_Sentiment_Flag"] = data_subset["clean_text"].apply(lambda x:1 if(len(re.findall(Negative_Complex_Pattern,x))>0) else 0)

In [94]:
data_subset["Positive_Sentiment_Flag"] = data_subset["clean_text"].apply(lambda x:1 if(len(re.findall(Positive_Complex_Pattern,x))>0) else 0)

In [96]:
data_subset

,text,airline_sentiment,clean_text,Negative_Sentiment_Flag,Positive_Sentiment_Flag
0,@VirginAmerica plus you've added commercials t...,positive,plus added commercials experience tacky,0,0
1,@VirginAmerica it's really aggressive to blast...,negative,really aggressive blast obnoxious entertainme...,0,0
2,@VirginAmerica and it's a really big bad thing...,negative,really big bad thing,0,0
3,@VirginAmerica seriously would pay $30 a fligh...,negative,seriously would pay flight seats playing it r...,0,0
4,"@VirginAmerica yes, nearly every time I fly VX...",positive,yes nearly every time fly vx ear worm wont go...,0,0
...,...,...,...,...,...
11536,@AmericanAir my flight was Cancelled Flightled...,negative,flight cancelled flightled leaving tomorrow m...,1,0
11537,@AmericanAir right on cue with the delays👌,negative,right cue delays,0,0
11538,@AmericanAir thank you we got on a different f...,positive,thank got different flight chicago,0,1
11539,@AmericanAir leaving over 20 minutes Late Flig...,negative,leaving minutes late flight warnings communic...,0,0


In [90]:
len(re.findall(Positive_Complex_Pattern,' plus added commercials experience tacky '))>0

False

In [104]:
X_train, X_test, y_train, y_test = train_test_split(data_subset[["clean_text","Negative_Sentiment_Flag","Positive_Sentiment_Flag"]],data_subset["airline_sentiment"],test_size=0.1,shuffle=True)

In [106]:
vectorizer=TfidfVectorizer(use_idf=True,max_features=10)
X_train_tfid=vectorizer.fit_transform(X_train["clean_text"])
X_test_tfid=vectorizer.fit_transform(X_test["clean_text"])

In [110]:
mydf = pd.DataFrame(X_train_tfid.toarray())
mydf_test = pd.DataFrame(X_test_tfid.toarray())

In [112]:
flag_columns_train = X_train[["Negative_Sentiment_Flag","Positive_Sentiment_Flag"]]
flag_columns_test = X_test[["Negative_Sentiment_Flag","Positive_Sentiment_Flag"]]

In [114]:
X_train_vectors_tfidf_combined_with_flag = pd.concat([mydf.reset_index(drop=True),flag_columns_train.reset_index(drop=True)],axis=1)

In [116]:
X_test_vectors_tfidf_combined_with_flag = pd.concat([mydf_test.reset_index(drop=True),flag_columns_test.reset_index(drop=True)],axis=1)

In [120]:
classifier=MultinomialNB()
classifier.fit(X_train_vectors_tfidf_combined_with_flag,y_train)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:


from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
cf_matrix=confusion_matrix(y_test,y_pred)
print(f'Confusion_matrix: {cf_matrix}')
print('**************************')
print('')
clf_report=classification_report(y_test,y_pred)
print(f'{clf_report}')
print('**************************')
print('')
Accuracy=accuracy_score(y_test,y_pred)
print(f'accuracy_score : {Accuracy}')
print('**************************')
print('')


In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve, auc

# Assuming 'y_test' contains true labels and 'y_pred_proba' contains predicted probabilities
y_pred_proba = classifier.predict_proba(X_test_tfid)[:, 1]  # Probability of positive class
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC score: {auc:.6f}")